In [ ]:
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
import brighteyes_ism.simulation.PSF_sim as sim
import brighteyes_ism.analysis.Graph_lib as gra
import brighteyes_ism.analysis.Tools_lib as tool
import brighteyes_ism.analysis.Deconv_lib as dec
import brighteyes_ism.analysis.FRC_lib as frc
import brighteyes_ism.simulation.Tubulin_sim as simTub
import lib as flim
import napari as nap
from skimage.filters import gaussian
path = '/home/lcuneo/Documents/PostDoc/Bufalino/Code/'
os.chdir(path)
from s2ISM.src.s2ism import s2ism as s2
from scipy.signal import convolve

In [ ]:
# %% Set Plot parameters for figures
plt.close('all')

plt.rcParams['ps.fonttype'] = 42
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

In [ ]:
wd_path = os.getcwd()
files_path = os.path.join(wd_path, 'files')
if not os.path.exists(files_path):
    os.makedirs(files_path)

In [ ]:
# %% psf
b = np.finfo(float).eps

exPar = sim.simSettings()
exPar.na = 1.4   # numerical aperture
exPar.wl = 640   # wavelength [nm]
exPar.gamma = 45  # parameter describing the light polarization
exPar.beta = 90  # parameter describing the light polarization
exPar.n = 1.5
exPar.mask_sampl = 151

emPar = exPar.copy()
emPar.wl = 670

grid = sim.GridParameters()
grid.N = 5
grid.Nz = 1
grid.Nx = 1250
grid.pxsizex = 50 * 1e3 / grid.Nx
grid.pxsizez = 720
grid.pxpitch = 75e3
grid.pxdim = 50e3


file_psf = os.path.join(files_path, 'PSFs.npy') 

if os.path.exists(file_psf):
    psf = np.load(file_psf)
else:
    psf = sim.SPAD_PSF_3D(grid, exPar, emPar,stack ='positive')[0]
    np.save(file_psf, psf)


In [ ]:
#%% Temporal functions
def rect(t, w):
    r = np.where(abs(t) <= w/2, 1, 0)*1.
    r /= r.sum()
    return r


def irf(t, w):
    r = np.where(abs(t-w/2) <= w/2, 1, 0)
    return r


def exp_decay(tau, t):
    decay = np.exp(-t/tau)
    decay[t < 0] = 0
    return decay


def smooth_irf(t, s, w):
    irf_1 = rect(t, w)
    # g = gauss(t, 1, 0, t[len(t)//2], s)
    # irf_2 = convolve(irf_1, g, mode='same')
    dt = t[1] - t[0]
    irf_2 = gaussian(irf_1, s/dt)
    irf_2 /= irf_2.sum()
    return irf_2

In [ ]:
psf = np.squeeze(psf)
psf_fin = np.repeat(psf, 3*nbin).reshape((grid.Nx, grid.Nx, 25, 3*nbin))

irf_fin = np.empty((ntau,nbin*3))

# %% Temporal decay
dt = 2*1e3  # 0.4 ns
nbin = 81
window = 2.1  # ns
time = np.arange(3*nbin)*dt
t0 = time[len(time)//2]
# t0 = nbin*dt + 0
par = np.array([0.3, 2])
#irf = smooth_irf(time - t0, *par)

irf = h5.load(os.path.join(files_path, 'data-10-04-2024-19-50-34.h5')) #correggere
irf = irf['data'].sum((0,1,2,3))

ntau = 2

tau_1 = 2.8  # ns
tau_2 = 3.4  # ns

period = nbin*dt
decay_1 = 0
decay_2 = 0

for tc in [-1, 0, 1]:
    decay_1 += exp_decay(tau_1, time-nbin*dt-tc*period)
    decay_2 += exp_decay(tau_2, time-nbin*dt-tc*period)


decay_w = np.stack((decay_1, decay_2))
del decay_1, decay_2
for i in range(ntau):
    irf_fin[i] = convolve(decay_w[i], irf,mode='same')


psf_fin = np.einsum('ijkl , ml -> ijkml', psf_fin, irf_fin)

del irf_fin
irf = irf[nbin:-nbin]

for i in range(ntau):
    psf_fin[:,:,:,i]/=psf_fin[:,:,:,i].sum()
    
psf_fin = psf_fin[..., nbin:-nbin]
decay_1 = decay_w[0,..., nbin:-nbin]
decay_2 = decay_w[1,..., nbin:-nbin]

#%%
psf_fin = np.swapaxes(psf_fin,-3,-1)  #(x y t tau nch)
psf_fin = np.swapaxes(psf_fin,0,-2)  #(tau y t x nch)
psf_fin = np.swapaxes(psf_fin,-3,-2)  #(tau y x t nch)
psf_fin = np.swapaxes(psf_fin,1,2)  #(tau x y t nch)

In [ ]:
files_path_tau = os.path.join(files_path, r'tau{}_tau{}'.format(tau_1, tau_2))

if not os.path.exists(files_path_tau):
    os.makedirs(files_path_tau)

In [ ]:
dataset = h5.load(os.path.join(files_path, 'data-10-04-2024-19-27-47.h5')) #correggere
dataset = dataset['data']

In [ ]:
# %% Confocal case
data_clsm = dataset.sum(-1)
psf_clsm = psf_fin.sum(-1)
data_clsm = data_clsm[..., None]
psf_clsm = psf_clsm[..., None]

# %%
n_iter = 200

file_rec = os.path.join(files_path_tau, 'confocal_recon_unmixing n_iter{}.npy'.format(n_iter))

if os.path.exists(file_rec):
    test = np.load(file_rec)
else:
    test = s2.max_likelihood_reconstruction(data_clsm, psf_clsm , max_iter=n_iter, rep_to_save ='all', initialization = 'sum')[0]
    np.save(file_rec, test)


In [ ]:
inv = -1

fig, (ax1, ax2) = plt.subplots(1, 2)
im1 = ax1.imshow(test[inv,0])
ax1.set_title('First reconstruction')
fig.colorbar(im1, orientation='vertical')
im2=ax2.imshow(test[inv,1])
ax2.set_title('Second reconstruction')
fig.colorbar(im2, orientation='vertical')

In [ ]:
# %% ISM case
n_iter = 200

file_rec = os.path.join(files_path_tau,  'ISM_recon_unmixing n_iter{}.npy'.format(n_iter))

if os.path.exists(file_rec):
   test_ism = np.load(file_rec)
else:
    test_ism = s2.max_likelihood_reconstruction(dataset, psf_fin,
                                            max_iter=n_iter, rep_to_save ='all', initialization = 'flat')[0]
    np.save(file_rec, test_ism)


In [ ]:
inv = -1

fig, (ax1, ax2) = plt.subplots(1, 2)
im1 = ax1.imshow(test_ism[inv,0])
ax1.set_title('First reconstruction')
fig.colorbar(im1, orientation='vertical')
im2=ax2.imshow(test_ism[inv,1])
ax2.set_title('Second reconstruction')
fig.colorbar(im2, orientation='vertical')